In [1]:
#DataFrame input output

In [3]:
df = pd.DataFrame({"name":list("ABC"),"age":range(20,23)})

In [4]:
df

,name,age
0,A,20
1,B,21
2,C,22


In [19]:
df.to_csv("test.csv", index=False)

In [20]:
!ls

3_10__pandas.ipynb              3_12__강사님자료copy.ipynb
3_10__pandas_dataframe.ipynb    datas
3_10__pandas_summary.ipynb      pandas2.ipynb
3_12__pandas_dataframe IO.ipynb test.csv


In [21]:
!cat test.csv

name,age
A,20
B,21
C,22


In [22]:
pd.read_csv("test.csv")

,name,age
0,A,20
1,B,21
2,C,22


In [23]:
#csv파일 저장시 데이터에 ,가 있으면 에러가 발생할 수 있다.
#tsv 파일로 저장 가능

In [26]:
df.to_csv("test.tsv",index=False,sep="\t")

In [27]:
!cat test.tsv

name	age
A	20
B	21
C	22


In [28]:
#excel로 데이터 저장, 불러오기
# conda install -c anaconda xlrd openpyxl xlsxwriter

In [29]:
df.to_excel("test.xlsx",sheet_name="test",engine="xlsxwriter",encoding="utf-8-sig")

In [30]:
!ls

3_10__pandas.ipynb              pandas2.ipynb
3_10__pandas_dataframe.ipynb    test.csv
3_10__pandas_summary.ipynb      test.tsv
3_12__pandas_dataframe IO.ipynb test.txv
3_12__강사님자료copy.ipynb      test.xlsx
datas


In [31]:
pd.read_excel("test.xlsx","test")

,Unnamed: 0,name,age
0,0,A,20
1,1,B,21
2,2,C,22


## pandas dataframe pivot
- 데이터프레임의 특정 컬럼데이터를 index, column,value를 선택해서 데이터 프레임을 만드는 방법
- 간단한 기술통계 분석을 할 수 있습니다.

In [32]:
df = pd.read_csv("datas/train.csv")

In [33]:
df.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [34]:
# 성별과 좌석등급에 따라 승객의 수 확인

In [35]:
#groupby, pivot

In [36]:
df1 = df.groupby(["Sex","Pclass"]).size()

In [37]:
df1

Sex     Pclass
female  1          94
        2          76
        3         144
male    1         122
        2         108
        3         347
dtype: int64

In [53]:
df1 = df.groupby(["Sex","Pclass"]).size().reset_index(name="Count")

In [54]:
df1

,Sex,Pclass,Count
0,female,1,94
1,female,2,76
2,female,3,144
3,male,1,122
4,male,2,108
5,male,3,347


In [55]:
df1 = df1.pivot("Sex","Pclass","Count")

In [56]:
df1

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


In [ ]:
#pivot table

In [58]:
df["Count"]=1

In [67]:
df.pivot_table("Count","Sex","Pclass",aggfunc=np.sum)

Survived,0,1
Sex,,
female,81,233
male,468,109


In [62]:
#quiz : 성별에 따른 생존자수

In [87]:
result_df = df.pivot_table("Count","Sex","Survived",aggfunc=np.sum)

In [71]:
#total 칼럼, total 인덱스를 추가

In [88]:
result_df

Survived,0,1
Sex,,
female,81,233
male,468,109


In [84]:
result_df[0]

Sex
female     81
male      468
Name: 0, dtype: int64

In [85]:
result_df[1]

Sex
female    233
male      109
Name: 1, dtype: int64

In [86]:
result_df = result_df[0] + result_df[1]

In [89]:
# np.sum(result_df,axis = -1)

In [91]:
result_df["total"] = np.sum(result_df.values,axis=1)

In [92]:
result_df

Survived,0,1,total
Sex,,,
female,81,233,314
male,468,109,577


In [93]:
result_df.loc["total"] = np.sum(result_df.values,axis=0)

In [94]:
result_df

Survived,0,1,total
Sex,,,
female,81,233,314
male,468,109,577
total,549,342,891


In [95]:
#pivot_table 에서 NaN데이터 처리

In [99]:
round(result_df / len(df) * 100,2)

Survived,0,1,total
Sex,,,
female,9.09,26.15,35.24
male,52.53,12.23,64.76
total,61.62,38.38,100.00


In [100]:
df3 = df.pivot_table("Count","Survived",["Parch","Pclass"],aggfunc=np.sum)

In [101]:
df3

Parch        0                  1                 2                3       \
Pclass       1     2      3     1     2     3     1     2     3    2    3   
Survived                                                                    
0         64.0  86.0  295.0  10.0   8.0  35.0   5.0   3.0  32.0  NaN  2.0   
1         99.0  48.0   86.0  21.0  24.0  20.0  16.0  13.0  11.0  2.0  1.0   

Parch       4         5    6  
Pclass      1    3    3    3  
Survived                      
0         1.0  3.0  4.0  1.0  
1         NaN  NaN  1.0  NaN

In [102]:
df3 = df.pivot_table("Count","Survived",["Parch","Pclass"],aggfunc=np.sum, fill_value=0)

In [103]:
df3

Parch      0            1           2          3     4     5  6
Pclass     1   2    3   1   2   3   1   2   3  2  3  1  3  3  3
Survived                                                       
0         64  86  295  10   8  35   5   3  32  0  2  1  3  4  1
1         99  48   86  21  24  20  16  13  11  2  1  0  0  1  0

In [106]:
result = df.pivot_table("Count","Survived",["Parch","Pclass"],aggfunc=np.sum, fill_value=0, dropna=False)

In [112]:
result

Parch      0            1           2          3        4        5        6  \
Pclass     1   2    3   1   2   3   1   2   3  1  2  3  1  2  3  1  2  3  1   
Survived                                                                      
0         64  86  295  10   8  35   5   3  32  0  0  2  1  0  3  0  0  4  0   
1         99  48   86  21  24  20  16  13  11  0  2  1  0  0  0  0  0  1  0   

Parch           
Pclass    2  3  
Survived        
0         0  1  
1         0  0

In [108]:
#pandas dataframe row,column 출력 갯수 설정

In [113]:
pd.options.display.max_rows = 900

60

In [111]:
pd.options.display.max_columns = 30